In [8]:
import pandas as pd

global_weekly = pd.read_csv("all-weeks-global.csv")
most_popular = pd.read_csv("most-popular.csv")
all_weeks_by_country = pd.read_csv("all-weeks-countries.csv")
netflix_full = pd.read_csv("netflix_full.csv")
shows_all = pd.read_csv("shows_all.csv")
movies_all = pd.read_csv("movies_all.csv")

In [9]:
global_weekly.head()

,week,category,weekly_rank,show_title,season_title,weekly_hours_viewed,cumulative_weeks_in_top_10
0,2022-08-21,Films (English),1,Day Shift,NaN,63390000,2
1,2022-08-21,Films (English),2,Look Both Ways,NaN,48060000,1
2,2022-08-21,Films (English),3,The Next 365 Days,NaN,39310000,1
3,2022-08-21,Films (English),4,Purple Hearts,NaN,23410000,4
4,2022-08-21,Films (English),5,Untold: The Girlfriend Who Didn't Exist,NaN,20050000,1


In [10]:
most_popular.head()

,category,rank,show_title,season_title,hours_viewed_first_28_days
0,Films (English),1,Red Notice,NaN,364020000
1,Films (English),2,Don't Look Up,NaN,359790000
2,Films (English),3,Bird Box,NaN,282020000
3,Films (English),4,The Gray Man,NaN,253830000
4,Films (English),5,The Adam Project,NaN,233160000


In [11]:
all_weeks_by_country.head()

,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10
0,Argentina,AR,2022-08-21,Films,1,Look Both Ways,NaN,1
1,Argentina,AR,2022-08-21,Films,2,Day Shift,NaN,2
2,Argentina,AR,2022-08-21,Films,3,Bank Robbers: The Last Great Heist,NaN,2
3,Argentina,AR,2022-08-21,Films,4,The Next 365 Days,NaN,1
4,Argentina,AR,2022-08-21,Films,5,The Angry Birds Movie 2,NaN,1


In [12]:
netflix_full.head()

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Ariel,movie,"Comedy, Crime, Drama",1988.0,tt0094675,7.4,9226.0,NaN
1,Shadows in Paradise,movie,"Comedy, Drama, Music",1986.0,tt0092149,7.4,8054.0,NaN
2,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2387285.0,NaN
3,American Beauty,movie,Drama,1999.0,tt0169547,8.3,1251348.0,NaN
4,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997.0,tt0119116,7.6,528407.0,NaN


In [13]:
shows_all.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,0,1,Breaking Bad,2008,18+,9.4/10,100/100,1,0,0,0,1
1,1,2,Stranger Things,2016,16+,8.7/10,96/100,1,0,0,0,1
2,2,3,Attack on Titan,2013,18+,9.0/10,95/100,1,1,0,0,1
3,3,4,Better Call Saul,2015,18+,8.8/10,94/100,1,0,0,0,1
4,4,5,Dark,2017,16+,8.8/10,93/100,1,0,0,0,1


In [14]:
movies_all.head()

,Unnamed: 0,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,0,1,The Irishman,2019,18+,98/100,1,0,0,0,0
1,1,2,Dangal,2016,7+,97/100,1,0,0,0,0
2,2,3,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,0
3,3,4,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,0
4,4,5,Roma,2018,18+,94/100,1,0,0,0,0


In [15]:
import pandas as pd
import re

def clean_top10_df(df):
  
    df["category"] = df["category"].astype(str).str.strip()

    cat_re  = re.compile(r"^\s*([A-Za-z]+)")   
    lang_re = re.compile(r"\(([^)]+)\)")       

    
    new_cat, new_lang = [], []

    for val in df["category"]:
        # Extract type word
        m = cat_re.search(val)
        type_word = m.group(1).lower() if m else ""
        new_cat.append("Movie" if type_word.startswith("film") else "TV")

        
        l = lang_re.search(val)
        new_lang.append(l.group(1).title() if l else "")

    
    df["Category"] = new_cat
    df["Language"] = new_lang

    
    df.drop(columns=["category"], inplace=True)


    def season_logic(row):
        if row["Category"] == "Movie":
            return "S0", 0
        txt = str(row["season_title"]) if pd.notna(row["season_title"]) else ""
        m = re.search(r"(\d+)", txt)
        num = int(m.group(1)) if m else 1
        return str(num), num

    season_vals = df.apply(season_logic, axis=1)
    df["season_title"], df["season_num"] = zip(*season_vals)

    return df



global_weekly        = clean_top10_df(global_weekly)
most_popular         = clean_top10_df(most_popular)
all_weeks_by_country = clean_top10_df(all_weeks_by_country)

print(" Added 'Category' + 'Language', cleaned seasons, dropped old 'category'.")


 Added 'Category' + 'Language', cleaned seasons, dropped old 'category'.


In [16]:
import pandas as pd
import re

def finalize_season_title(df):
   
    def _new_season(row):
        if row["Category"] == "Movie":
            return "S0"
        # TV show → extract first number in the string; default = 1
        txt = str(row["season_title"]) if pd.notna(row["season_title"]) else ""
        m = re.search(r"(\d+)", txt)
        num = int(m.group(1)) if m else 1
        return f"S{num}"

    # Build the cleaned column
    df["season_title"] = df.apply(_new_season, axis=1)
    
    # Drop season_num if it exists
    if "season_num" in df.columns:
        df.drop(columns=["season_num"], inplace=True)
    
    return df



global_weekly        = finalize_season_title(global_weekly)
most_popular         = finalize_season_title(most_popular)
all_weeks_by_country = finalize_season_title(all_weeks_by_country)

print(" season_title standardized (S0 for movies, S# for TV) and season_num removed.")


 season_title standardized (S0 for movies, S# for TV) and season_num removed.


In [17]:

unique_vals = all_weeks_by_country["Language"].dropna().unique()   # NumPy array of uniques
print(unique_vals)

all_weeks_by_country.drop(columns=["Language"], inplace=True) 

['']


In [18]:
global_weekly.head()

,week,weekly_rank,show_title,season_title,weekly_hours_viewed,cumulative_weeks_in_top_10,Category,Language
0,2022-08-21,1,Day Shift,S0,63390000,2,Movie,English
1,2022-08-21,2,Look Both Ways,S0,48060000,1,Movie,English
2,2022-08-21,3,The Next 365 Days,S0,39310000,1,Movie,English
3,2022-08-21,4,Purple Hearts,S0,23410000,4,Movie,English
4,2022-08-21,5,Untold: The Girlfriend Who Didn't Exist,S0,20050000,1,Movie,English


In [19]:
most_popular.head()

,rank,show_title,season_title,hours_viewed_first_28_days,Category,Language
0,1,Red Notice,S0,364020000,Movie,English
1,2,Don't Look Up,S0,359790000,Movie,English
2,3,Bird Box,S0,282020000,Movie,English
3,4,The Gray Man,S0,253830000,Movie,English
4,5,The Adam Project,S0,233160000,Movie,English


In [20]:
all_weeks_by_country.head()

,country_name,country_iso2,week,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10,Category
0,Argentina,AR,2022-08-21,1,Look Both Ways,S0,1,Movie
1,Argentina,AR,2022-08-21,2,Day Shift,S0,2,Movie
2,Argentina,AR,2022-08-21,3,Bank Robbers: The Last Great Heist,S0,2,Movie
3,Argentina,AR,2022-08-21,4,The Next 365 Days,S0,1,Movie
4,Argentina,AR,2022-08-21,5,The Angry Birds Movie 2,S0,1,Movie


In [21]:
import pandas as pd
import numpy as np
import re


In [22]:
netflix_full = pd.read_csv("netflix_full.csv")
shows_all    = pd.read_csv("shows_all.csv")
movies_all   = pd.read_csv("movies_all.csv")


In [23]:
for df in (netflix_full, shows_all, movies_all):
    if "Unnamed: 0" in df.columns:
        df.drop(columns=["Unnamed: 0"], inplace=True)


In [24]:
netflix_full["Title_key"] = netflix_full["title"].str.strip().str.lower()
shows_all["Title_key"]    = shows_all["Title"].str.strip().str.lower()
movies_all["Title_key"]   = movies_all["Title"].str.strip().str.lower()


In [25]:

netflix_full["releaseYear"] = (
    pd.to_numeric(netflix_full["releaseYear"], errors="coerce")
      .astype("Int64")
)



In [26]:
imdb_cols = netflix_full[["Title_key", "imdbAverageRating", "imdbNumVotes"]]

movies_enriched = movies_all.merge(imdb_cols, on="Title_key", how="left")


In [27]:

shows_all["imdbAverageRating"] = (
    shows_all["IMDb"].astype(str).str.extract(r"([\d.]+)")[0].astype(float)
)
shows_all.drop(columns=["IMDb"], inplace=True)

shows_enriched = shows_all.merge(imdb_cols, on="Title_key", how="left")


In [28]:
type_map = {0: "Movie", 1: "TV"}
for df in (movies_enriched, shows_enriched):
    df["Category"] = df["Type"].map(type_map)


In [29]:
netflix_full.to_csv("netflix_full_clean.csv",   index=False, na_rep="")
movies_enriched.to_csv("movies_all_enriched.csv", index=False, na_rep="")
shows_enriched.to_csv("shows_all_enriched.csv",  index=False, na_rep="")

print(" All three cleaned CSVs written to disk")


 All three cleaned CSVs written to disk


In [30]:
netflix_full.head()

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,Title_key
0,Ariel,movie,"Comedy, Crime, Drama",1988,tt0094675,7.4,9226.0,NaN,ariel
1,Shadows in Paradise,movie,"Comedy, Drama, Music",1986,tt0092149,7.4,8054.0,NaN,shadows in paradise
2,Forrest Gump,movie,"Drama, Romance",1994,tt0109830,8.8,2387285.0,NaN,forrest gump
3,American Beauty,movie,Drama,1999,tt0169547,8.3,1251348.0,NaN,american beauty
4,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997,tt0119116,7.6,528407.0,NaN,the fifth element


In [31]:
movies_enriched.head()

,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Title_key,imdbAverageRating,imdbNumVotes,Category
0,1,The Irishman,2019,18+,98/100,1,0,0,0,0,the irishman,7.8,454459.0,Movie
1,2,Dangal,2016,7+,97/100,1,0,0,0,0,dangal,NaN,NaN,Movie
2,3,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,0,david attenborough: a life on our planet,8.9,35438.0,Movie
3,4,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,0,lagaan: once upon a time in india,8.1,124641.0,Movie
4,5,Roma,2018,18+,94/100,1,0,0,0,0,roma,7.6,174556.0,Movie


In [32]:
shows_enriched.head()

,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Title_key,imdbAverageRating_x,imdbAverageRating_y,imdbNumVotes,Category
0,1,Breaking Bad,2008,18+,100/100,1,0,0,0,1,breaking bad,9.4,9.5,2343003.0,TV
1,2,Stranger Things,2016,16+,96/100,1,0,0,0,1,stranger things,8.7,8.6,1452622.0,TV
2,3,Attack on Titan,2013,18+,95/100,1,1,0,0,1,attack on titan,9.0,9.1,616671.0,TV
3,4,Better Call Saul,2015,18+,94/100,1,0,0,0,1,better call saul,8.8,9.0,737210.0,TV
4,5,Dark,2017,16+,93/100,1,0,0,0,1,dark,8.8,8.7,494434.0,TV


In [33]:

netflix_full["releaseYear"] = (
    pd.to_numeric(netflix_full["releaseYear"], errors="coerce")
      .astype("Int64")
)

dupe_pairs = [("imdbAverageRating", "_x", "_y"),
              ("imdbNumVotes",      "_x", "_y")]

for base, x, y in dupe_pairs:
    col_x = f"{base}{x}"
    col_y = f"{base}{y}"
    
    if col_x in shows_enriched.columns and col_y in shows_enriched.columns:
        # Use the parsed _x value first, fallback to _y
        shows_enriched[base] = shows_enriched[col_x].fillna(shows_enriched[col_y])
        shows_enriched.drop(columns=[col_x, col_y], inplace=True)
    elif col_x in shows_enriched.columns:
        shows_enriched.rename(columns={col_x: base}, inplace=True)
    elif col_y in shows_enriched.columns:
        shows_enriched.rename(columns={col_y: base}, inplace=True)
    # else: nothing to do, column already unique



netflix_full.to_csv("netflix_full_clean.csv",    index=False, na_rep="")
movies_enriched.to_csv("movies_all_enriched.csv", index=False, na_rep="")
shows_enriched.to_csv("shows_all_enriched.csv",   index=False, na_rep="")

print(" Cleaned & saved:\n"
      "   • netflix_full_clean.csv\n"
      "   • movies_all_enriched.csv\n"
      "   • shows_all_enriched.csv")


 Cleaned & saved:
   • netflix_full_clean.csv
   • movies_all_enriched.csv
   • shows_all_enriched.csv


In [34]:
netflix_full.head()

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,Title_key
0,Ariel,movie,"Comedy, Crime, Drama",1988,tt0094675,7.4,9226.0,NaN,ariel
1,Shadows in Paradise,movie,"Comedy, Drama, Music",1986,tt0092149,7.4,8054.0,NaN,shadows in paradise
2,Forrest Gump,movie,"Drama, Romance",1994,tt0109830,8.8,2387285.0,NaN,forrest gump
3,American Beauty,movie,Drama,1999,tt0169547,8.3,1251348.0,NaN,american beauty
4,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997,tt0119116,7.6,528407.0,NaN,the fifth element


In [35]:
movies_enriched.head()

,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Title_key,imdbAverageRating,imdbNumVotes,Category
0,1,The Irishman,2019,18+,98/100,1,0,0,0,0,the irishman,7.8,454459.0,Movie
1,2,Dangal,2016,7+,97/100,1,0,0,0,0,dangal,NaN,NaN,Movie
2,3,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,0,david attenborough: a life on our planet,8.9,35438.0,Movie
3,4,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,0,lagaan: once upon a time in india,8.1,124641.0,Movie
4,5,Roma,2018,18+,94/100,1,0,0,0,0,roma,7.6,174556.0,Movie


In [36]:
shows_enriched.head()

,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Title_key,imdbNumVotes,Category,imdbAverageRating
0,1,Breaking Bad,2008,18+,100/100,1,0,0,0,1,breaking bad,2343003.0,TV,9.4
1,2,Stranger Things,2016,16+,96/100,1,0,0,0,1,stranger things,1452622.0,TV,8.7
2,3,Attack on Titan,2013,18+,95/100,1,1,0,0,1,attack on titan,616671.0,TV,9.0
3,4,Better Call Saul,2015,18+,94/100,1,0,0,0,1,better call saul,737210.0,TV,8.8
4,5,Dark,2017,16+,93/100,1,0,0,0,1,dark,494434.0,TV,8.8


In [37]:
netflix_full["releaseYear"] = (
    pd.to_numeric(netflix_full["releaseYear"], errors="coerce")
      .astype("Int64")
)
netflix_full.to_csv("netflix_full_clean.csv", index=False, na_rep="")


In [38]:
import pandas as pd


def finalise_top10_df(df):
    
    # ── key for joins
    df["show_title_key"] = df["show_title"].str.strip().str.lower()

    # ── week to datetime when present
    if "week" in df.columns:
        df["week"] = pd.to_datetime(df["week"], errors="coerce")

    # ── convert typical numeric columns if they exist
    num_cols = ["weekly_rank",
                "weekly_hours_viewed",
                "cumulative_weeks_in_top_10",
                "hours_viewed_first_28_days"]
    for col in num_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    return df



global_weekly_clean        = finalise_top10_df(global_weekly.copy())
most_popular_clean         = finalise_top10_df(most_popular.copy())
all_weeks_by_country_clean = finalise_top10_df(all_weeks_by_country.copy())



global_weekly_clean.to_csv(       "global_weekly_clean.csv",        index=False, na_rep="")
most_popular_clean.to_csv(        "most_popular_clean.csv",         index=False, na_rep="")
all_weeks_by_country_clean.to_csv("all_weeks_by_country_clean.csv", index=False, na_rep="")

print(" Saved three Tableau-ready files:")
print("   • global_weekly_clean.csv")
print("   • most_popular_clean.csv")
print("   • all_weeks_by_country_clean.csv")


 Saved three Tableau-ready files:
   • global_weekly_clean.csv
   • most_popular_clean.csv
   • all_weeks_by_country_clean.csv
